In [ ]:
print("문제 4-2 : 조건부 분기가 있는 메뉴 추천 시스템 ( LangGraph 사용)")

In [ ]:
import os
import re
from typing import List
from langchain_core.messages import AIMessage, HumanMessage, BaseMessage
from langchain_community.vectorstores import FAISS
from langchain_upstage import UpstageEmbeddings
from langchain_core.documents import Document
from langgraph.graph.message import MessageState
from langgraph.graph import StateGraph, END

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
UPSTAGE_API_KEY = os.getenv("UPSTAGE_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
print(OPENAI_API_KEY[:2])
print(UPSTAGE_API_KEY[30:])
print(TAVILY_API_KEY[:2])

In [ ]:
llm = ChatUpstage(
        model="solar-pro",
        base_url="https://api.upstage.ai/v1",
        temperature=0.5
    )
print(llm)

# from langchain_openai import ChatOpenAI
# llm = ChatOpenAI(
#     model='gpt-3.5-turbo-0125', 
#     temperature=0,
# )
# print(llm.model_name)

In [ ]:
# 데이터 로드 및 분할

loader = TextLoader("./data/cafe_menu.txt", encoding="utf-8")

documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=50)
docs = text_splitter.split_documents(documents)

# 벡터 DB 생성 및 저장

embeddings = UpstageEmbeddings(model="solar-embedding-1-large")

db = FAISS.from_documents(docs, embeddings)

db.save_local("./db/cafe_db_upstage")

print("벡터 DB가 './db/cafe_db_upstage' 폴더에 성공적으로 생성 및 저장되었습니다.")

In [ ]:
# Vector DB 문서에서 구조화된 정보 추출하는 함수
def parse_menu_doc(doc: Document) -> dict:
    """Vector DB 문서에서 'Key: Value' 형식의 정보를 추출합니다."""
    info = {}
    for line in doc.page_content.split('\n'):
        if ':' in line:
            key, value = line.split(':', 1)
            info[key.strip()] = value.strip()
    return info

# 질문 유형을 분류하는 함수 (Router)
def classify_question(state: MessageState) -> str:
    """사용자의 마지막 메시지를 기반으로 질문 유형을 분류합니다."""
    user_message = state["messages"][-1].content.lower()
    
    price_keywords = ["가격", "얼마"]
    recommend_keywords = ["추천", "어떤게", "뭐가 맛있", "인기"]
    
    if any(keyword in user_message for keyword in price_keywords):
        return "price_inquiry"
    elif any(keyword in user_message for keyword in recommend_keywords):
        return "recommendation_request"
    else:
        return "menu_inquiry"

# 각 문의 유형을 처리하는 노드(Node) 함수들
def handle_menu_inquiry(state: MessageState) -> dict:
    """메뉴 관련 문의를 처리합니다."""
    user_message = state["messages"][-1].content
    # 의미론적 검색: 사용자 메시지 직접 활용
    docs = menu_db.similarity_search(user_message, k=1)
    
    if not docs:
        response = "죄송합니다, 문의하신 메뉴를 찾을 수 없습니다."
    else:
        info = parse_menu_doc(docs[0])
        response = (
            f"✅ {info.get('메뉴', '이름 없음')}에 대한 정보입니다:\n"
            f" - 가격: {info.get('가격', '정보 없음')}\n"
            f" - 재료: {info.get('재료', '정보 없음')}\n"
            f" - 설명: {info.get('설명', '정보 없음')}"
        )
    return {"messages": [AIMessage(content=response)]}

def handle_price_inquiry(state: MessageState) -> dict:
    """가격 관련 문의를 처리합니다."""
    user_message = state["messages"][-1].content
    # 사용자 메시지로 검색하여 가장 관련성 높은 메뉴의 가격을 보여줌
    docs = menu_db.similarity_search(user_message, k=1)
    
    if not docs:
        response = "죄송합니다, 문의하신 메뉴의 가격 정보를 찾을 수 없습니다."
    else:
        info = parse_menu_doc(docs[0])
        response = f"✅ {info.get('메뉴', '해당 메뉴')}의 가격은 {info.get('가격', '정보 없음')}입니다."
    
    return {"messages": [AIMessage(content=response)]}

def handle_recommendation_request(state: MessageState) -> dict:
    """메뉴 추천 요청을 처리합니다."""
    user_message = state["messages"][-1].content
    # 사용자 메시지 + 기본 추천 키워드로 검색
    docs = menu_db.similarity_search(user_message, k=3)
    if not docs:
        docs = menu_db.similarity_search("인기 있는 메뉴", k=3)

    response_parts = ["✅ 이런 메뉴는 어떠세요?"]
    for doc in docs:
        info = parse_menu_doc(doc)
        response_parts.append(f" - **{info.get('메뉴')}**: {info.get('설명')}")
        
    return {"messages": [AIMessage(content="\n".join(response_parts))]}


In [ ]:
# LangGraph 그래프 생성 

# MessageState를 상태로 사용하는 그래프 정의
graph = StateGraph(MessageState)

# 노드 추가
graph.add_node("menu_inquiry", handle_menu_inquiry)
graph.add_node("price_inquiry", handle_price_inquiry)
graph.add_node("recommendation_request", handle_recommendation_request)

# 시작점 설정
graph.set_entry_point("classify_question")

# 조건부 엣지(Edge) 설정: 분류 결과에 따라 다음 노드로 분기
graph.add_conditional_edges(
    "classify_question",
    classify_question,
    {
        "menu_inquiry": "menu_inquiry",
        "price_inquiry": "price_inquiry",
        "recommendation_request": "recommendation_request",
    },
)

# 각 처리 노드 이후에는 그래프를 종료
graph.add_edge("menu_inquiry", END)
graph.add_edge("price_inquiry", END)
graph.add_edge("recommendation_request", END)

# 그래프 컴파일
app = graph.compile()

def run_chatbot(question: str, thread_id: str):
    """지정된 스레드에서 챗봇을 실행하고 응답을 출력합니다."""
    print(f"\n👤 User: {question}")
    
    # 대화 이력을 유지하기 위해 thread_id 사용
    config = {"configurable": {"thread_id": thread_id}}
    
    # HumanMessage 객체로 입력
    input_message = [HumanMessage(content=question)]
    
    # 스트리밍 방식으로 응답 받기
    response_stream = app.stream(input=input_message, config=config)
    
    full_response = ""
    print("🤖 AI: ", end="")
    for chunk in response_stream:
        # 마지막 단계의 AI 메시지만 출력
        last_message = next(iter(chunk.values()))["messages"][-1]
        if isinstance(last_message, AIMessage):
            print(last_message.content, end="")
            full_response += last_message.content
    print()


# 테스트 실행
thread_id = "cafe-thread-1" # 대화 세션을 식별하는 ID
run_chatbot("안녕하세요!", thread_id)
run_chatbot("콜드브루에 대해 알려주세요.", thread_id)
run_chatbot("치즈케이크는 얼마인가요?", thread_id)
run_chatbot("달달한 음료 추천해줘.", thread_id)